In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Running batch inference using `tfrecord` files

This notebook demonstrates how to run batch inference on data in `tfrecord` format. 



In [ ]:
# Reloads modules automatically before executing any code/script
%load_ext autoreload
%autoreload 2

### Import libraries

Import the required libraries.

In [ ]:
import os
import time
from datetime import datetime
import pandas as pd

# Import the Vertex AI SDK for Python
from google.cloud import aiplatform as vertex_ai

Import [`utils.py`](utils.py), which is a
custom script that has utility functions to streamline configuration and
submission of a Vertex AI custom job and to track the generated artifacts and
metrics from the custom job.

In [ ]:
import utils

### Configure environment settings

Based on the [environment setup](README.md) you did previously, configure the
following environment settings:

-  `PROJECT_ID`: Configure the Google Cloud project ID.
-  `REGION`: Configure the
    [region](https://cloud.google.com/vertex-ai/docs/general/locations) to use
    for Vertex AI operations throughout this notebook.
-  `BUCKET`: Configure the Google Cloud Storage bucket name that you created
    during environment setup. Vertex AI uses this bucket for operations such as
    staging the code and saving generated artifacts.
-  `TENSORBOARD_NAME`: Configure the managed TensorBoard instance name that
    you created during the environment setup.

In [ ]:
# Project definitions
PROJECT_ID = '<YOUR PROJECT ID>' # Change to your project id.
REGION = '<YOUR REGION>'  # Change to your region.

# Bucket definitions
BUCKET = '<YOUR BUCKET NAME>' # Change to your bucket.

In [ ]:
# Tensorboard definitions
TENSORBOARD_NAME = '<YOUR TENSORBOARD NAME>' # Change to your Tensorboard instance name

In [ ]:
# Get Vertex AI TensorBoard ID based on name
TENSORBOARD_ID = ! gcloud ai tensorboards list --filter="displayName={TENSORBOARD_NAME}" --format="value(name)" --region={REGION} 2>/dev/null 
TENSORBOARD_ID = TENSORBOARD_ID[0]

print(f"TENSORBOARD_ID = {TENSORBOARD_ID}")

In [ ]:
# Configure the custom container image name
IMAGE_NAME = 't5x-base' # Change to your image name
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'

Verify that the image exists in Container Registry:

In [ ]:
! gcloud container images describe $IMAGE_URI

### Configure experiment settings


In [ ]:
EXPERIMENT_NAME = '<YOUR EXPERIMENT NAME>' # Change to your experiment name

EXPERIMENT_WORKSPACE = f'gs://{BUCKET}/experiments/{EXPERIMENT_NAME}'
EXPERIMENT_RUNS = f'{EXPERIMENT_WORKSPACE}/runs'

### Initialize the Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project, bucket, and
experiment.

In [ ]:
vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=EXPERIMENT_WORKSPACE,
    experiment=EXPERIMENT_NAME
)

---

## Configure and run a batch inference job



### Configure a Gin file for the batch inference job


In [ ]:
GIN_FILE = '../configs/infer_t511_base_tr_en.gin'
! cat {GIN_FILE}

### Configure the Vertex AI custom job

In [ ]:
INFER_RUN_NAME = f'<YOUR RUN NAME>' # Change to your run name for the custom job
INFER_RUN_ID = f'{EXPERIMENT_NAME}-{INFER_RUN_NAME}-{datetime.now().strftime("%Y%m%d%H%M")}'
INFER_RUN_DIR = f'{EXPERIMENT_RUNS}/{INFER_RUN_ID}'

Specify the run mode as `infer` to run the T5X launch script in inference
mode.

In [ ]:
RUN_MODE = 'infer'

#### Select the checkpoint

In [ ]:
# Get all experiment runs and run directories
utils.get_all_experiment_run_directories(EXPERIMENT_NAME)

In [ ]:
! gsutil ls '<YOUR PREVIOUS RUN DIRECTORY>' # Change to the previous run directory

In [ ]:
CHECKPOINT_PATH = '<YOUR MODEL CHECKPOINT PATH>' # Change to the path where model checkpoint exists

Configure runtime parameters for the inference.

In [ ]:
GIN_FILES = [GIN_FILE]
GIN_OVERWRITES = [
    'USE_CACHED_TASKS=False',
    f'CHECKPOINT_PATH="{CHECKPOINT_PATH}"',
    f'INFER_OUTPUT_DIR="{INFER_RUN_DIR}"',
    f"TF_EXAMPLE_FILE_PATHS=['gs://<YOUR PATH>/eval.tfrecords']"
]

To help with troubleshooting, display the values of local variables.

In [ ]:
for key in [
    "PROJECT_ID", "REGION", "BUCKET", "TENSORBOARD_NAME", "TENSORBOARD_ID", 
    "IMAGE_NAME", "IMAGE_URI", 
    "EXPERIMENT_NAME", "EXPERIMENT_WORKSPACE", "EXPERIMENT_RUNS", 
    "GIN_FILES", "GIN_OVERWRITES", 
    "INFER_RUN_NAME", "INFER_RUN_ID", "INFER_RUN_DIR", "RUN_MODE",
    "CHECKPOINT_PATH"
]:
    print(f"{key}={eval(key)}")

### Create the Vertex AI custom job

Configure the worker pool specification based on TPU availability in the region.
See
[Vertex AI locations](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)
for details.

In [ ]:
# Define worker pool resource specification
# Machine to run the custom job on. For TPUs, use `cloud-tpu`.
MACHINE_TYPE = 'cloud-tpu'
# Accelerator type to attach to the machine. For TPUs, use `TPU_V2`, `TPU_V3`.
ACCELERATOR_TYPE = 'TPU_V2'
# Number of accelerators to attach to the machine.
# For TPUs, specify the number of cores to be provisioned.
ACCELERATOR_COUNT = 8

Create the custom job spec.

In [ ]:
job = utils.create_t5x_custom_job(
    display_name=INFER_RUN_ID,
    machine_type=MACHINE_TYPE,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=ACCELERATOR_COUNT,
    image_uri=IMAGE_URI,
    run_mode=RUN_MODE,
    gin_files=GIN_FILES,
    model_dir=CHECKPOINT_PATH,
    gin_overwrites=GIN_OVERWRITES
)

In [ ]:
job.job_spec

### Submit the custom job to Vertex AI and track the experiment

The `submit_and_track_tx5_vertex_job` function launches the T5X script in
inference mode with the fine-tuned model checkpoint specification. The function
submits the job to Vertex AI, which generates inferences results and writes them
to the Cloud Storage bucket. To monitor progress of the job, click the URL in
the cell output. The URL goes to the Vertex AI Custom Job console for the job.

In [ ]:
utils.submit_and_track_t5x_vertex_job(
    custom_job=job,
    job_display_name=INFER_RUN_ID,
    run_name=INFER_RUN_ID,
    experiment_name=EXPERIMENT_NAME,
    execution_name=INFER_RUN_ID,
    model_dir=INFER_RUN_DIR,
    vertex_ai=vertex_ai,
    run_mode=RUN_MODE
)

### Explore batch inference results

The batch inference job writes inference results to the run directory that you
configured. The output is written in JSON lines (`.jsonl`) format.

In [ ]:
! gsutil ls $INFER_RUN_DIR

The following example is a snippet from the output of the batch inference job
for the translation task from a fine-tuned
[WMT English-to-German translation](https://www.tensorflow.org/datasets/catalog/wmt_t2t_translate)
model.

``` json
{
	"inputs": {
		"inputs_pretokenized": "translate English to German: As recently as last Tuesday, the Nasdaq indices were not calculated for one hour due to data transfer errors.",
		"targets_pretokenized": "Erst am Dienstag waren die Indizes der Nasdaq wegen Fehlern im Datentransport eine Stunde lang nicht berechnet worden."
	},
	"prediction": "Erst am vergangenen Dienstag wurden die Nasdaq-Indexe wegen Daten\u00fcbertragungen f\u00fcr eine Stunde nicht berechnet."
}
```

---

## What's next?

This notebook template covers how to run a T5X job for fine-tuning, evaluation,
and inference tasks on Vertex AI.

You can copy this notebook template for a specific task or dataset or
configuration and then make changes as needed to run the T5X job on Vertex AI.
Refer to other notebooks in the
[notebooks directory](/) of this repo as
inspiration.